In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
from dotenv import load_dotenv
import os


In [2]:
#API KEYS
load_dotenv()

bls_api = os.getenv("bls_api")
bea_api = os.getenv("bea_api")


In [3]:
#URLS

url_bea = "https://apps.bea.gov/api/data"
url_bls = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

In [4]:
#Datos para demsempleo por Estado
estados = {
    "Alabama": "LAUST010000000000003",
    "Alaska": "LAUST020000000000003",
    "Arizona": "LAUST040000000000003",
    "Arkansas": "LAUST050000000000003",
    "California": "LAUST060000000000003",
    "Colorado": "LAUST080000000000003",
    "Connecticut": "LAUST090000000000003",
    "Delaware": "LAUST100000000000003",
    "Florida": "LAUST120000000000003",
    "Georgia": "LAUST130000000000003",
    "Hawaii": "LAUST150000000000003",
    "Idaho": "LAUST160000000000003",
    "Illinois": "LAUST170000000000003",
    "Indiana": "LAUST180000000000003",
    "Iowa": "LAUST190000000000003",
    "Kansas": "LAUST200000000000003",
    "Kentucky": "LAUST210000000000003",
    "Louisiana": "LAUST220000000000003",
    "Maine": "LAUST230000000000003",
    "Maryland": "LAUST240000000000003",
    "Massachusetts": "LAUST250000000000003",
    "Michigan": "LAUST260000000000003",
    "Minnesota": "LAUST270000000000003",
    "Mississippi": "LAUST280000000000003",
    "Missouri": "LAUST290000000000003",
    "Montana": "LAUST300000000000003",
    "Nebraska": "LAUST310000000000003",
    "Nevada": "LAUST320000000000003",
    "New Hampshire": "LAUST330000000000003",
    "New Jersey": "LAUST340000000000003",
    "New Mexico": "LAUST350000000000003",
    "New York": "LAUST360000000000003",
    "North Carolina": "LAUST370000000000003",
    "North Dakota": "LAUST380000000000003",
    "Ohio": "LAUST390000000000003",
    "Oklahoma": "LAUST400000000000003",
    "Oregon": "LAUST410000000000003",
    "Pennsylvania": "LAUST420000000000003",
    "Rhode Island": "LAUST440000000000003",
    "South Carolina": "LAUST450000000000003",
    "South Dakota": "LAUST460000000000003",
    "Tennessee": "LAUST470000000000003",
    "Texas": "LAUST480000000000003",
    "Utah": "LAUST490000000000003",
    "Vermont": "LAUST500000000000003",
    "Virginia": "LAUST510000000000003",
    "Washington": "LAUST530000000000003",
    "West Virginia": "LAUST540000000000003",
    "Wisconsin": "LAUST550000000000003",
    "Wyoming": "LAUST560000000000003"
}

In [5]:
meses = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

In [6]:
rangos = [(1990,1999),(2000, 2009), (2010, 2019), (2020, 2025)]

In [7]:
#Payload
registros = []

series = list(estados.values())

for start, end in rangos:
    for i in range(0, len(series), 25):  
        bloque = series[i:i+25]
        payload = {
            "registrationKey": bls_api,
            "seriesid": bloque,
            "startyear": str(start),
            "endyear": str(end)
        }
        url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
        response = requests.post(url, json=payload)
        data = response.json()

        # Procesar cada serie
        for serie in data["Results"]["series"]:
            estado = [k for k, v in estados.items() if v == serie["seriesID"]][0]
            for d in serie["data"]:
                if d["periodName"] in meses:
                    registros.append({
                        "Estado": estado,
                        "Año": int(d["year"]),
                        "Mes": meses[d["periodName"]],
                        "TasaDesempleo": float(d["value"])
                    })
        time.sleep(0.5)  # pequeño delay para no saturar la API


In [8]:
df = pd.DataFrame(registros)
df["Fecha"] = pd.to_datetime(df["Año"].astype(str) + "-" + df["Mes"].astype(str).str.zfill(2))
df = df.sort_values(["Estado", "Fecha"]).reset_index(drop=True)
df = df[df['Estado'] != 'New York city']
df = df[df['Estado'] != 'Los Angeles County']
df = df[['Estado','Fecha','TasaDesempleo']]
df['Trim'] = df['Fecha'].dt.to_period('Q').astype(str)
df

,Estado,Fecha,TasaDesempleo,Trim
0,Alabama,1990-01-01,7.1,1990Q1
1,Alabama,1990-02-01,7.3,1990Q1
2,Alabama,1990-03-01,6.6,1990Q1
3,Alabama,1990-04-01,6.4,1990Q2
4,Alabama,1990-05-01,6.3,1990Q2
...,...,...,...,...
21395,Wyoming,2025-04-01,3.0,2025Q2
21396,Wyoming,2025-05-01,3.2,2025Q2
21397,Wyoming,2025-06-01,3.7,2025Q2
21398,Wyoming,2025-07-01,2.8,2025Q3


In [9]:
#Parametros para GDP

params = {
    "UserID": bea_api,
    "method": "GetData",
    "datasetname": "Regional",
    "TableName": "SQGDP1",   
    "Frequency": "Q",
    "Year": "ALL",    
    'GeoFips':'STATE',     
    "ResultFormat": "JSON",
    'LineCode':1
}

In [10]:
#Request GDP
response = requests.get(url_bea, params=params)
response.status_code == 200

True

In [11]:
gdp = response.json()
gdp

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': '1877DB8B-6730-4F80-B187-242ACB99D57C'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SQGDP1'},
    {'ParameterName': 'FREQUENCY', 'ParameterValue': 'Q'},
    {'ParameterName': 'YEAR', 'ParameterValue': 'ALL'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '1'}]},
  'Results': {'Statistic': 'Real Gross Domestic Product (GDP)',
   'UnitOfMeasure': 'Millions of chained 2017 dollars',
   'PublicTable': 'SQGDP1 State quarterly gross domestic product (GDP) summary',
   'UTCProductionTime': '2025-11-06T13:59:21.657',
   'NoteRef': '1',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'Da

In [12]:
records = gdp['BEAAPI']['Results']['Data']
records

[{'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q1',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '15844727.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q2',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '15922782.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q3',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '16047587.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2005Q4',
  'CL_UNIT': 'Millions of chained 2017 dollars',
  'UNIT_MULT': '6',
  'DataValue': '16136734.0',
  'NoteRef': '1'},
 {'Code': 'SQGDP1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2006Q1',
  'CL_UNIT': 'Millions of chaine

In [13]:
gdp_data = pd.DataFrame(records)[['GeoName','TimePeriod','DataValue']]
gdp_data[gdp_data['GeoName']=='United States']
gdp_data

,GeoName,TimePeriod,DataValue
0,United States,2005Q1,15844727.0
1,United States,2005Q2,15922782.0
2,United States,2005Q3,16047587.0
3,United States,2005Q4,16136734.0
4,United States,2006Q1,16353835.0
...,...,...,...
4915,Far West,2024Q2,4606652.1
4916,Far West,2024Q3,4649588.3
4917,Far West,2024Q4,4679775.7
4918,Far West,2025Q1,4674104.5


In [14]:
gdp_data["DataValue"] = pd.to_numeric(gdp_data["DataValue"], errors="coerce")
gdp_data.rename(columns={'GeoName':'Estado','TimePeriod':'Trim','DataValue':'GDP'}, inplace=True)
gdp_data

,Estado,Trim,GDP
0,United States,2005Q1,15844727.0
1,United States,2005Q2,15922782.0
2,United States,2005Q3,16047587.0
3,United States,2005Q4,16136734.0
4,United States,2006Q1,16353835.0
...,...,...,...
4915,Far West,2024Q2,4606652.1
4916,Far West,2024Q3,4649588.3
4917,Far West,2024Q4,4679775.7
4918,Far West,2025Q1,4674104.5


In [15]:
base_cox = pd.merge(
    df,
    gdp_data[['Estado','Trim','GDP']],
    on=['Estado','Trim'],
    how='left'
)
base_cox

,Estado,Fecha,TasaDesempleo,Trim,GDP
0,Alabama,1990-01-01,7.1,1990Q1,NaN
1,Alabama,1990-02-01,7.3,1990Q1,NaN
2,Alabama,1990-03-01,6.6,1990Q1,NaN
3,Alabama,1990-04-01,6.4,1990Q2,NaN
4,Alabama,1990-05-01,6.3,1990Q2,NaN
...,...,...,...,...,...
21395,Wyoming,2025-04-01,3.0,2025Q2,39982.7
21396,Wyoming,2025-05-01,3.2,2025Q2,39982.7
21397,Wyoming,2025-06-01,3.7,2025Q2,39982.7
21398,Wyoming,2025-07-01,2.8,2025Q3,NaN


In [16]:
#Parametros Personla Income
params_pi = {
    "UserID": bea_api,
    "method": "GetData",
    "datasetname": "Regional",
    "TableName": "SQINC1",   
    "Frequency": "Q",
    "Year": "ALL",    
    'GeoFips':'STATE',     
    "ResultFormat": "JSON",
    'LineCode':1
}

In [17]:
#Request 
response_pi = requests.get(url_bea, params=params_pi)
response_pi.status_code == 200

True

In [18]:
pi = response_pi.json()
pi

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': '1877DB8B-6730-4F80-B187-242ACB99D57C'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SQINC1'},
    {'ParameterName': 'FREQUENCY', 'ParameterValue': 'Q'},
    {'ParameterName': 'YEAR', 'ParameterValue': 'ALL'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '1'}]},
  'Results': {'Statistic': 'Personal income',
   'UnitOfMeasure': 'Millions of dollars',
   'PublicTable': 'SQINC1 State quarterly personal income summary: personal income, population, per capita personal income',
   'UTCProductionTime': '2025-11-06T13:59:24.340',
   'NoteRef': ' ',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': '

In [19]:
records_pi = pi['BEAAPI']['Results']['Data']
records_pi

[{'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q1',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '204641.7'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q2',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '210069.4'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q3',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '215368.6'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1948Q4',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '215900.3'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '1949Q1',
  'CL_UNIT': 'Millions of dollars',
  'UNIT_MULT': '6',
  'DataValue': '209582.6'},
 {'Code': 'SQINC1-1',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'Tim

In [20]:
data_pi = pd.DataFrame(records_pi)[['GeoName','TimePeriod','DataValue']]
data_pi

,GeoName,TimePeriod,DataValue
0,United States,1948Q1,204641.7
1,United States,1948Q2,210069.4
2,United States,1948Q3,215368.6
3,United States,1948Q4,215900.3
4,United States,1949Q1,209582.6
...,...,...,...
18595,Far West *,2024Q2,4739480.1
18596,Far West *,2024Q3,4788860.9
18597,Far West *,2024Q4,4876382.7
18598,Far West *,2025Q1,4932645.0


In [21]:
data_pi['DataValue'] = pd.to_numeric(data_pi['DataValue'],errors='coerce')
data_pi.rename(columns={'GeoName':'Estado','TimePeriod':'Trim','DataValue':'Personal_Income'}, inplace=True)
data_pi["Estado"] = data_pi["Estado"].str.replace("*", "", regex=False).str.strip()
data_pi


,Estado,Trim,Personal_Income
0,United States,1948Q1,204641.7
1,United States,1948Q2,210069.4
2,United States,1948Q3,215368.6
3,United States,1948Q4,215900.3
4,United States,1949Q1,209582.6
...,...,...,...
18595,Far West,2024Q2,4739480.1
18596,Far West,2024Q3,4788860.9
18597,Far West,2024Q4,4876382.7
18598,Far West,2025Q1,4932645.0


In [22]:
base_cox = pd.merge(
    base_cox,
    data_pi[['Estado','Trim','Personal_Income']],
    on=['Estado','Trim'],
    how='left'
)

In [23]:
#Separacion por zonas
state_abbr = {
    "Alabama":"AL", "Alaska":"AK", "Arizona":"AZ", "Arkansas":"AR", "California":"CA",
    "Colorado":"CO", "Connecticut":"CT", "Delaware":"DE", "DistrictOfColumbia":"DC",
    "Florida":"FL", "Georgia":"GA", "Hawaii":"HI", "Idaho":"ID", "Illinois":"IL",
    "Indiana":"IN", "Iowa":"IA", "Kansas":"KS", "Kentucky":"KY", "Louisiana":"LA",
    "Maine":"ME", "Maryland":"MD", "Massachusetts":"MA", "Michigan":"MI",
    "Minnesota":"MN", "Mississippi":"MS", "Missouri":"MO", "Montana":"MT",
    "Nebraska":"NE", "Nevada":"NV", "NewHampshire":"NH", "NewJersey":"NJ",
    "NewMexico":"NM", "NewYork":"NY", "NorthCarolina":"NC", "NorthDakota":"ND",
    "Ohio":"OH", "Oklahoma":"OK", "Oregon":"OR", "Pennsylvania":"PA",
    "RhodeIsland":"RI", "SouthCarolina":"SC", "SouthDakota":"SD", "Tennessee":"TN",
    "Texas":"TX", "Utah":"UT", "Vermont":"VT", "Virginia":"VA", "Washington":"WA",
    "WestVirginia":"WV", "Wisconsin":"WI", "Wyoming":"WY"
}



In [24]:
base_cox['Abr'] = base_cox['Estado'].map(state_abbr)
#base_cox['region'] = base_cox['Abr'].map(state_region_map)
base_cox

,Estado,Fecha,TasaDesempleo,Trim,GDP,Personal_Income,Abr
0,Alabama,1990-01-01,7.1,1990Q1,NaN,62823.1,AL
1,Alabama,1990-02-01,7.3,1990Q1,NaN,62823.1,AL
2,Alabama,1990-03-01,6.6,1990Q1,NaN,62823.1,AL
3,Alabama,1990-04-01,6.4,1990Q2,NaN,64011.2,AL
4,Alabama,1990-05-01,6.3,1990Q2,NaN,64011.2,AL
...,...,...,...,...,...,...,...
21395,Wyoming,2025-04-01,3.0,2025Q2,39982.7,52880.9,WY
21396,Wyoming,2025-05-01,3.2,2025Q2,39982.7,52880.9,WY
21397,Wyoming,2025-06-01,3.7,2025Q2,39982.7,52880.9,WY
21398,Wyoming,2025-07-01,2.8,2025Q3,NaN,NaN,WY


In [25]:
base_cox['U_3m'] = base_cox.groupby('Estado')['TasaDesempleo'].transform(lambda x: x.rolling(3, min_periods = 3).mean().round(2))
base_cox['U_min12']= base_cox.groupby('Estado')['U_3m'].transform(lambda x: x.rolling(12, min_periods = 12).min().round(2))
base_cox['SahmRule'] = (base_cox['U_3m']-base_cox['U_min12']).round(2)
base_cox['Dummy'] = (base_cox['SahmRule'] >=0.5).astype(int)
base_cox['Fecha'] = pd.PeriodIndex(base_cox['Trim'], freq='Q').to_timestamp()
base_cox

,Estado,Fecha,TasaDesempleo,Trim,GDP,Personal_Income,Abr,U_3m,U_min12,SahmRule,Dummy
0,Alabama,1990-01-01,7.1,1990Q1,NaN,62823.1,AL,NaN,NaN,NaN,0
1,Alabama,1990-01-01,7.3,1990Q1,NaN,62823.1,AL,NaN,NaN,NaN,0
2,Alabama,1990-01-01,6.6,1990Q1,NaN,62823.1,AL,7.00,NaN,NaN,0
3,Alabama,1990-04-01,6.4,1990Q2,NaN,64011.2,AL,6.77,NaN,NaN,0
4,Alabama,1990-04-01,6.3,1990Q2,NaN,64011.2,AL,6.43,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
21395,Wyoming,2025-04-01,3.0,2025Q2,39982.7,52880.9,WY,3.47,2.97,0.50,1
21396,Wyoming,2025-04-01,3.2,2025Q2,39982.7,52880.9,WY,3.30,2.97,0.33,0
21397,Wyoming,2025-04-01,3.7,2025Q2,39982.7,52880.9,WY,3.30,2.97,0.33,0
21398,Wyoming,2025-07-01,2.8,2025Q3,NaN,NaN,WY,3.23,2.97,0.26,0


In [26]:
base_cox

,Estado,Fecha,TasaDesempleo,Trim,GDP,Personal_Income,Abr,U_3m,U_min12,SahmRule,Dummy
0,Alabama,1990-01-01,7.1,1990Q1,NaN,62823.1,AL,NaN,NaN,NaN,0
1,Alabama,1990-01-01,7.3,1990Q1,NaN,62823.1,AL,NaN,NaN,NaN,0
2,Alabama,1990-01-01,6.6,1990Q1,NaN,62823.1,AL,7.00,NaN,NaN,0
3,Alabama,1990-04-01,6.4,1990Q2,NaN,64011.2,AL,6.77,NaN,NaN,0
4,Alabama,1990-04-01,6.3,1990Q2,NaN,64011.2,AL,6.43,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
21395,Wyoming,2025-04-01,3.0,2025Q2,39982.7,52880.9,WY,3.47,2.97,0.50,1
21396,Wyoming,2025-04-01,3.2,2025Q2,39982.7,52880.9,WY,3.30,2.97,0.33,0
21397,Wyoming,2025-04-01,3.7,2025Q2,39982.7,52880.9,WY,3.30,2.97,0.33,0
21398,Wyoming,2025-07-01,2.8,2025Q3,NaN,NaN,WY,3.23,2.97,0.26,0


In [27]:
base_cox = base_cox[base_cox['Fecha']>='2007-10-01']
base_cox = base_cox[base_cox['Fecha']<='2009-06-01']
base_cox['GDP'] = np.log(base_cox['GDP'])
base_cox['Personal_Income']=np.log(base_cox['Personal_Income'])
base_cox =  base_cox[['Estado','Trim','Fecha','TasaDesempleo','Personal_Income','GDP','SahmRule','Dummy']]
base_cox.reset_index(drop=True)

,Estado,Trim,Fecha,TasaDesempleo,Personal_Income,GDP,SahmRule,Dummy
0,Alabama,2007Q4,2007-10-01,3.9,11.944742,12.223257,0.53,1
1,Alabama,2007Q4,2007-10-01,4.1,11.944742,12.223257,0.46,0
2,Alabama,2007Q4,2007-10-01,4.4,11.944742,12.223257,0.56,1
3,Alabama,2008Q1,2008-01-01,5.1,11.951368,12.218838,0.96,1
4,Alabama,2008Q1,2008-01-01,5.0,11.951368,12.218838,1.26,1
...,...,...,...,...,...,...,...,...
1045,Wyoming,2009Q1,2009-01-01,5.4,10.165944,10.693339,2.13,1
1046,Wyoming,2009Q1,2009-01-01,5.9,10.165944,10.693339,2.83,1
1047,Wyoming,2009Q2,2009-04-01,5.8,10.125350,10.618523,3.03,1
1048,Wyoming,2009Q2,2009-04-01,5.9,10.125350,10.618523,3.20,1


In [28]:
# Agrupar por Estado y Trimestre, dejando un registro por trimestre
base_cox = (base_cox
    .groupby(['Estado', 'Trim'], as_index=False)
    .agg({
        'GDP':'mean',
        'TasaDesempleo': 'mean',
        'Personal_Income': 'mean',
        'SahmRule': 'mean',
        'Dummy': 'max'
    })
)

# Volver a generar la columna de fecha
base_cox['Fecha'] = pd.PeriodIndex(base_cox['Trim'], freq='Q').to_timestamp()

In [29]:
base_cox['recession_start'] = (
    (base_cox['Dummy'] == 1) &
    ((base_cox['Dummy'].shift(1) == 0) | (base_cox['Estado'] != base_cox['Estado'].shift(1)))
)
base_cox

,Estado,Trim,GDP,TasaDesempleo,Personal_Income,SahmRule,Dummy,Fecha,recession_start
0,Alabama,2007Q4,12.223257,4.133333,11.944742,0.516667,1,2007-10-01,True
1,Alabama,2008Q1,12.218838,5.000000,11.951368,1.216667,1,2008-01-01,False
2,Alabama,2008Q2,12.223525,5.200000,11.996815,1.133333,1,2008-04-01,False
3,Alabama,2008Q3,12.215361,6.400000,11.968071,2.146667,1,2008-07-01,False
4,Alabama,2008Q4,12.185424,7.000000,11.958987,2.503333,1,2008-10-01,False
...,...,...,...,...,...,...,...,...,...
345,Wyoming,2008Q2,10.609667,2.666667,10.212244,0.623333,1,2008-04-01,False
346,Wyoming,2008Q3,10.624036,2.700000,10.216658,0.543333,1,2008-07-01,False
347,Wyoming,2008Q4,10.702215,3.300000,10.203903,0.456667,1,2008-10-01,False
348,Wyoming,2009Q1,10.693339,5.500000,10.165944,2.130000,1,2009-01-01,False


In [30]:
base_cox['recession_end'] = (
    (base_cox['Dummy'] == 0) &
    (base_cox['Dummy'].shift(1) == 1) &
    (base_cox['Estado'] == base_cox['Estado'].shift(1))
)
base_cox

,Estado,Trim,GDP,TasaDesempleo,Personal_Income,SahmRule,Dummy,Fecha,recession_start,recession_end
0,Alabama,2007Q4,12.223257,4.133333,11.944742,0.516667,1,2007-10-01,True,False
1,Alabama,2008Q1,12.218838,5.000000,11.951368,1.216667,1,2008-01-01,False,False
2,Alabama,2008Q2,12.223525,5.200000,11.996815,1.133333,1,2008-04-01,False,False
3,Alabama,2008Q3,12.215361,6.400000,11.968071,2.146667,1,2008-07-01,False,False
4,Alabama,2008Q4,12.185424,7.000000,11.958987,2.503333,1,2008-10-01,False,False
...,...,...,...,...,...,...,...,...,...,...
345,Wyoming,2008Q2,10.609667,2.666667,10.212244,0.623333,1,2008-04-01,False,False
346,Wyoming,2008Q3,10.624036,2.700000,10.216658,0.543333,1,2008-07-01,False,False
347,Wyoming,2008Q4,10.702215,3.300000,10.203903,0.456667,1,2008-10-01,False,False
348,Wyoming,2009Q1,10.693339,5.500000,10.165944,2.130000,1,2009-01-01,False,False


In [31]:
fecha_corte = pd.Timestamp("2009-06-30")
records = []

for state, group in base_cox.groupby('Estado'):
    group = group.reset_index(drop=True)
    start_date = None
    
    for i, row in group.iterrows():
        # Detecta inicio de recesión
        if row['recession_start']:
            start_date = row['Fecha']
            
        # Detecta final de recesión
        elif row['recession_end'] and start_date is not None:
            end_date = row['Fecha']
            if end_date > fecha_corte:
                end_date = fecha_corte
                event = 0
            else:
                event = 1
            duration = (end_date.year - start_date.year) * 4 + (end_date.quarter - start_date.quarter) # en meses
            records.append([state, duration, event])
            start_date = None

    # Si el estado sigue en recesión al final del período → censurado
    if start_date is not None:
        end_date = min(group['Fecha'].max(), fecha_corte)
        duration = (end_date.year - start_date.year) * 4 + (end_date.quarter - start_date.quarter)
        records.append([state, duration, 0])


In [32]:
df_cox = pd.DataFrame(records, columns=['id', 'duration', 'event'])
covariables = base_cox.groupby('Estado')[['GDP', 'Personal_Income','TasaDesempleo']].mean().reset_index()
df_cox = df_cox.merge(covariables, left_on='id', right_on='Estado', how='left').drop(columns='Estado')

df_cox

,id,duration,event,GDP,Personal_Income,TasaDesempleo
0,Alabama,6,0,12.201847,11.960909,6.514286
1,Alaska,6,0,10.849222,10.386709,6.914286
2,Arizona,6,0,12.599076,12.301745,6.414286
3,Arkansas,1,1,11.639422,11.429038,6.033333
4,Arkansas,3,0,11.639422,11.429038,6.033333
5,California,6,0,14.592066,14.268346,8.085714
6,Colorado,5,0,12.544146,12.240387,5.561905
7,Connecticut,5,0,12.520222,12.264476,6.104762
8,Delaware,5,0,11.110545,10.504757,5.533333
9,Florida,6,0,13.689441,13.479767,6.938095


In [33]:
%store base_cox

Stored 'base_cox' (DataFrame)


In [34]:
%store df_cox

Stored 'df_cox' (DataFrame)


In [35]:
state_region_map = {
    # --- NORTE ---
    "ME": "Norte", "NH": "Norte", "VT": "Norte", "MA": "Norte", "RI": "Norte", "CT": "Norte",
    "NY": "Norte", "NJ": "Norte", "PA": "Norte", "OH": "Norte", "MI": "Norte", "WI": "Norte",
    "MN": "Norte", "ND": "Norte", "SD": "Norte",

    # --- ESTE ---
    "DE": "Este", "MD": "Este", "DC": "Este", "VA": "Este", "WV": "Este", 
    "NC": "Este", "SC": "Este", "GA": "Este", "FL": "Este",

    # --- CENTRO ---
    "IL": "Centro", "IN": "Centro", "IA": "Centro", "MO": "Centro",
    "KS": "Centro", "NE": "Centro", "KY": "Centro", "TN": "Centro", "AR": "Centro",

    # --- SUR ---
    "AL": "Sur", "MS": "Sur", "LA": "Sur", "TX": "Sur", "OK": "Sur", 
    "NM": "Sur", "AZ": "Sur",

    # --- OESTE ---
    "CO": "Oeste", "UT": "Oeste", "NV": "Oeste", "CA": "Oeste", 
    "OR": "Oeste", "WA": "Oeste", "ID": "Oeste", "MT": "Oeste", "WY": "Oeste", "AK": "Oeste", "HI": "Oeste"

}